In [2]:
import numpy as np
import os
import json
import sys
import glob
import datetime
import cv2
import time
import h5py
import pickle

from skimage.morphology import white_tophat, disk, opening
from scipy import optimize
from scipy.ndimage.morphology import grey_closing
from scipy.signal import argrelextrema

# %qtconsole
%matplotlib qt5
%matplotlib auto
import matplotlib.pyplot as plt
from matplotlib import path
from matplotlib.patches import Rectangle

sys.path.append('/home/gravishlab/Documents/Python/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/Tracker/')
from Tracker.Tracker import Tracker


# stop telling me i have a nan in an array with a logical comparison
with np.errstate(invalid='ignore'):
    np.less([np.nan, 0], 1)

/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using matplotlib backend: Qt5Agg


In [13]:
%qtconsole

## DEFINE FUNCTIONS

In [5]:
# determine background image
def find_bkg(file,fr_sep):
    cap = cv2.VideoCapture(file)
    width = int(cap.get(3))
    height = int(cap.get(4))
    vid_length = np.min([int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),500])

    rgb_convert = np.array([[[0.2989]],[[0.5870]],[[0.1140]]]).T

    frames_to_read = np.arange(0,vid_length,fr_sep)
    frames = np.zeros((len(frames_to_read), height, width), np.uint8)
    for kk in range(0,len(frames_to_read)):
        fr_OI=frames_to_read[kk]
        cap.set(1,fr_OI)
        # Capture frame-by-frame
        ret, frame2 = cap.read()
        if ret:
            frames[kk,:,:] = np.sum(frame2*rgb_convert, axis = 2)
        else:
            print('couldnt get frame')
            return np.nan

    cap.release()
    img = np.mean(frames, axis = 0)
    return img

# make illumination even
def even_illumination(img, SE_size):
    SE = np.ones((SE_size,SE_size)).astype(bool)
    img_clos = grey_closing(img, structure = SE)
    img_bkg = img/img_clos
    return img_bkg

# use a hit-or-miss function to find horizontal and vertical edges, open to remove small blobs
def find_box_edges( img, line_wid, buffer_wid, hit_or_miss_cutoff):
    line_wid = 5
    buffer_wid = 10
    line_halfwid = int((line_wid-1)/2)
    (height,width)=img.shape
    
    # make hit-or-miss structuring element
    empty_SE = np.zeros(2*buffer_wid+line_wid).astype(bool)
    center_SE= empty_SE.copy()
    center_SE[buffer_wid:buffer_wid+line_wid]=True
    outer_SE = np.logical_not(center_SE)

    # run through each pixel to see if fits hit-or-miss criterion
    edges_raw = np.zeros(img.shape)
    for xx in range(buffer_wid+line_halfwid, width-(buffer_wid+line_halfwid)):
        for yy in range(buffer_wid+line_halfwid, height-(buffer_wid+line_halfwid)):
            pixOI = img[yy,xx]
            vtest = img[yy-(buffer_wid+line_halfwid):yy+(buffer_wid+line_halfwid)+1, xx]
            htest = img[yy, xx-(buffer_wid+line_halfwid):xx+(buffer_wid+line_halfwid)+1]
            for test in [vtest,htest]:
                center_mean = np.mean(test[center_SE])
                outer_mean = np.mean(test[outer_SE])
                if (outer_mean-center_mean>hit_or_miss_cutoff):
    #                 edges_raw[yy,xx]= outer_mean-center_mean 
                    edges_raw[yy,xx]= 1

    return edges_raw


In [6]:

# identify where lines are in image (assumes that the array is mostly aligned with camera axes)
def find_where_lines(opened, direction, plots_on = False, order=15):
    (height,width)=opened.shape
    if direction == 'vertical':
        sums=np.sum(opened,axis=0)
        xs = range(0,width)
    elif direction == 'horizontal':
        sums=np.sum(opened,axis=1)
        xs = range(0,height)
    else:
        print('provided direction not recognized')

    # take moving ave to smooth
    smoothed = np.convolve(sums, np.ones((5,))/5, mode='same')

    # find peaks and get rid of false positives
    peaks = argrelextrema(smoothed, np.greater_equal, order = order)[0]
    peaks = np.delete(peaks, np.where(smoothed[peaks]<100)) # get rid of low peaks
    peaks = np.delete(peaks, np.where(np.diff(peaks)<5)) # get rid of peaks with two equally tall points
    
    
    # plot things
    if plots_on:
        plt.close('all')
        plt.figure(figsize = (15,3))
        plt.plot(xs,sums, ':')
        plt.plot(xs,smoothed)
        plt.bar((peaks-search_wid), np.ones(len(peaks))*500, width = 2*search_wid, color = [0.6, 0.6, 0.6], alpha = 0.2, align = 'edge')
    return peaks


def fit_lines(img, img_edges, direction, lines, ax, search_wid):
#     opened = opening(img_edges, disk(2)) # open to remove small blobs
    peaks = find_where_lines(img_edges, direction, plots_on = False, order = 15)
    (height,width)=img.shape
    
    # what if not enough peaks? interpolate
    if len(peaks)<len(lines):
        where_gap = np.where(np.diff(peaks)>1.5*np.median(np.diff(peaks)))[0]
        n_unmarked_peaks = np.round(np.diff(peaks)/np.median(np.diff(peaks)))-1
        for gap in where_gap:
            peak_spacing = (peaks[gap+1]-peaks[gap])/(n_unmarked_peaks[gap]+1)
            new_peaks = np.arange(peaks[gap]+peak_spacing, peaks[gap+1], peak_spacing)
            peaks = np.insert(peaks, gap+1, np.round(new_peaks))
    elif len(peaks)>len(lines):
        if len(peaks) == 3:
            peak_to_remove  = 1
        else:
            peak_to_remove = np.argmin(np.diff(peaks)[1:-1])+1 # can't be first or last bc of lip on 3 and 5 mm
        peaks = np.delete(peaks, peak_to_remove)
    
    for pp,peak in enumerate(peaks):
        if direction == 'vertical':
            barOI = img_edges[:, peak+range(-1*search_wid,search_wid+1)]
            xs = np.arange(0,height)
        elif direction == 'horizontal':
#             print(peak+range(-1*search_wid,search_wid+1))
            barOI = img_edges[peak+range(-1*search_wid,search_wid+1), :].T
            
            xs = np.arange(0,width)
        else:
            print('provided direction not recognized')
            return
#         barOI = img_edges[:, peak+range(-1*search_wid,search_wid+1)]
        fg_locs = barOI*(np.arange(-10,11)+peak)
        fg_locs[fg_locs==0]=np.nan
        sums = np.nanmean(fg_locs,axis=1)
        where_interp =find_nan_gaps(sums, 5)
        if np.sum(where_interp)>0:
            sums = interp_vals(sums, find_interp_idcs(where_interp))
        smoothed = np.convolve(sums, np.ones((30,))/30, mode='same')

#         ax[1].plot(smoothed, range(0,550),'.b', alpha= 0.5, MarkerSize = 2)
#         plt.plot( sums, range(0,550), 'y.',alpha = 0.5, MarkerSize = 2)
#         plt.plot( smoothed, range(0,550),'.b', alpha= 0.8, MarkerSize = 2)


        xd = xs
        p = np.polyfit(xs[np.isfinite(smoothed)], smoothed[np.isfinite(smoothed)], 1)
        yd = p[1]+p[0]*xd
        if pp>=len(lines):
            return np.nan
        lines[pp,:]=np.flip(p, 0)
        if direction == 'vertical':
            ax[1].plot(smoothed, range(0,550),'.b', alpha= 0.5, MarkerSize = 2)
            ax[0].plot(yd,xd,':r')
            lines[pp,:]=np.array([-1*p[1]/p[0], 1/p[0]])
        elif direction == 'horizontal':
            ax[1].plot(range(0,1000), smoothed, '.b', alpha= 0.5, MarkerSize = 2)
            ax[0].plot(xd,yd,':r')
            lines[pp,:]=np.flip(p, 0)
    
    return lines
    
    
    
    


In [7]:
def find_nan_gaps(arr, limit):  
    from itertools import groupby
    yy = np.isnan(arr)
    xx = range(len(yy))
    where_gapOI = np.full(arr.shape, False)
    where_othergaps = np.full(arr.shape, False)
    for k,g in groupby(iter(xx), lambda x: yy[x]):
        if k == True: # if is a group of nan
            g = list(g)
            if any(x in g for x in [0, len(arr)-1]): # if first or last group
                where_othergaps[np.array(g)]=True
                continue       
            if len(g)<= limit: # length is below limit
                where_gapOI[np.array(g)]=True
    return where_gapOI
def find_interp_idcs(where_interpolate):
    interp_idcs = []
    for val in [-1,0,1]:
        interp_idcs = np.concatenate([interp_idcs,np.where(where_interpolate)[0]+val])
    interp_idcs = np.sort(np.array(list(set(interp_idcs)))) # get of repeat elements
    interp_idcs = interp_idcs[np.logical_and(interp_idcs>-1, interp_idcs < len(where_interpolate))].astype(np.uint32) # only elements in range
    return interp_idcs
def interp_vals(arr, interp_idcs): # array includes nan values
    temp = arr[interp_idcs]
    interpolated_vals = np.interp(
        interp_idcs, 
        interp_idcs[np.logical_not(np.isnan(temp))], temp[np.logical_not(np.isnan(temp))] )
    interp = arr.copy()
    interp[interp_idcs] = interpolated_vals
    return interp

def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x<x0], [lambda x:k1*x+y0-k1*x0, lambda x:k2*x+y0-k2*x0])


In [8]:
# find step heights

def get_step_heights(boxsize, img_raw, hlines, file):
    step_heights = np.full((int(np.ceil(16/boxsize)),int(np.ceil(30/boxsize))), 1)
    step_heights[::2,::2]=0
    step_heights[1::2,1::2]=0

    colony = file.split('/')[-3]

    if boxsize ==2:

        box_nums = np.arange(1,5)
        up = np.full(box_nums.shape, np.nan)
        for bb,box_num in enumerate(box_nums):
            # what are coordinates of box
            idcs0 = np.array([0,0,1,1])+box_num
            idcs1 = np.array([0,1,1,0])+box_num
            xs=(vlines[idcs1,0]-hlines[idcs0,0])/(hlines[idcs0,1]-vlines[idcs1,1])
            ys=hlines[idcs0,0]+hlines[idcs0,1]*xs

            left = int(np.mean(xs[[0,3]]))
            right = int(np.mean(xs[[1,2]]))
            bottom = int(np.mean(ys[[0,1]]))
            top = int(np.mean(ys[[2,3]]))

            left_pix_ave = np.mean(img_raw[bottom+5:top-5, left-3:left+3])
            right_pix_ave = np.mean(img_raw[bottom+5:top-5, right-3:right+3])

            if right_pix_ave-left_pix_ave > 5:
                up[bb]=0
            elif right_pix_ave-left_pix_ave < -5:
                up[bb]=1

        if np.sum(np.isnan(up))>2:
            print('error')
        elif np.sum(up==1):
            print('upper corner is step up')
            step_heights = step_heights*-1+1
        else:
            print('upper corner is step down')

        plt.figure()
        plt.imshow(img_raw, cmap='gray')
        plt.xlim([left-10, right+10])
        plt.ylim([top+10,bottom-10])
        rect=Rectangle((left, bottom), right-left, top-bottom, facecolor='none', edgecolor = 'r')
        plt.gca().add_patch(rect)
        plt.figure()
        plt.plot(img_raw[int(np.mean([top,bottom])), left-10:right+10])

    elif (boxsize in [3,5]) and (colony != 'Tunnel_20180508-09'):
        half_row_on_bottom = (np.argmin(np.diff(hlines[:,0]))==0)
#         print('Is partial row on bottom? %i'%half_row_on_bottom)
        if half_row_on_bottom:
            step_heights[0,:]=1
        else:
            step_heights[-1,:]=1 

    return step_heights



# Calculate and save box lines and step height profiles for each colony/substrate

In [9]:
vid_locations = '/media/gravishlab/SeagateExpansionDrive/AntTrack/'

folder_list = []

# searches for folders
# folder_list = glob.glob(os.path.join(vid_locations, 'Tunnel**/*[0-9]mm/'))
folder_list = glob.glob(os.path.join(vid_locations, 'Tunnel_2019**/*[!s]/'))

folder_list = sorted(folder_list)

print('Total Number of Videos: ',len(folder_list))
                


Total Number of Videos:  20


In [22]:
# load a few frames and take ave to get a good image of background w/o ants
# for tunnel 22-23 0 mm: file = 103, bkg frame sep = 50, hit_or_miss_cutoff = 5

plt.close('all')
for folder in folder_list[8:9]:
    file_list = glob.glob(os.path.join(folder, '*.mp4'))
    file = file_list[0]
    colony = file.split('/')[-3]
    subtype = folder.split('/')[-2]
    print('%s -- %s :'%(colony, file.split('/')[-2]))
    
    # initialize variables to save
    n_hlines = 2
    if subtype =='Step':
        n_vlines = 3
    elif subtype =='Gap':
        n_vlines = 4
    elif subtype =='Flat':
        n_vlines = 2
    elif subtype =='Array':
        boxsize = 3
        n_vlines = int(np.ceil(30/boxsize)+1)
        n_hlines = int(np.ceil(16/boxsize)+1)
        

    vlines = np.full((n_vlines,2), np.nan)
    hlines = np.full((n_hlines,2), np.nan)
    
    # get ready for plotting
    plt.figure(figsize = (12,5))
    
    ax =[]
    ax.append( plt.subplot(1,2,1))
    ax.append( plt.subplot(1,2,2))

    # load a few frames from video and average to determine good background
    img_raw = find_bkg(file, 100)
    ax[0].imshow(img_raw, cmap = 'gray')
    ax[0].axis('off')
    plt.title('%s -- %s :'%(file.split('/')[-3], file.split('/')[-2]))
    print('-- calculated background image')
    
    # find edges of boxes - if using raw img, cutoff = 7, line_wid = 5
    edges_raw = find_box_edges(img_raw, line_wid = 5, buffer_wid = 10,  hit_or_miss_cutoff=7)
    ax[1].imshow(edges_raw, cmap = 'gray')
    plt.axis('off')
    print('-- detected edges')
    
    # calculate slope of best fit lines to the edges
    search_wid = 10 # how far around each line to look at to find best fit line
    hlines = fit_lines(img_raw, edges_raw, 'horizontal', hlines, ax, search_wid)
    vlines = fit_lines(img_raw, edges_raw, 'vertical', vlines, ax, search_wid)
    print('-- fit lines to edges')
    
    
#     # calculate step heights?
#     if np.any(np.isfinite(hlines)) and np.any(np.isfinite(vlines)):
    
#         # calculate array step heights based on box size and looking at image
#         if  == 0:
#             step_heights = np.array([1])
#         else:
#             step_heights = get_step_heights(boxsize, img_raw, hlines, file)

#         # save coeff and slope as pickle
#         sname = ('/').join(file.split('/')[:-1]) + '/%s_%imm_Step_Height.pkl'%(colony, boxsize)
#         with open(sname, 'wb') as f:
#             pickle.dump([hlines, vlines, step_heights, img_raw], f)
#         f.close()
#         print('-- saved variable to pickle')
#     else:
#         print('-- error with line detection!!')
        
        
    # don't calculate step heights?
    if np.any(np.isfinite(hlines)) and np.any(np.isfinite(vlines)):

        # save coeff and slope as pickle
        sname = ('/').join(file.split('/')[:-1]) + '/%s_%s_Step_Height.pkl'%(colony, subtype)
        with open(sname, 'wb') as f:
            pickle.dump([hlines, vlines, img_raw], f)
        f.close()
        print('-- saved variable to pickle')
    else:
        print('-- error with line detection!!')
        
        


Tunnel_20190718 -- Array :
-- calculated background image
-- detected edges
-- fit lines to edges
-- saved variable to pickle


/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: Mean of empty slice
/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:80: RankWarning: Polyfit may be poorly conditioned


### find lines on the currently loaded trial, enables plotting to see what's going wrong

In [13]:
plt.close('all')
# direction = 'horizontal'
direction = 'vertical'
peaks= find_where_lines(edges_raw, direction, plots_on = True, order = 15)


## Old methods that don't work

In [60]:
# detect local min
from skimage.morphology import extrema

h = 0.05
mins = extrema.h_minima(img, h)
plt.figure()
plt.subplot(1,2,1)
plt.imshow(img, cmap = 'gray')
plt.subplot(1,2,2)
plt.imshow(mins, cmap = 'gray')

In [141]:
# # FIND INTERSECTIONS USING STRUCTURING ELEMENTS

# SE_rad = 15
# spoke_wid = 5
# spoke_halfwid = int((spoke_wid-1)/2)
# empty_SE = np.zeros((2*SE_rad+1,2*SE_rad+1)).astype(bool)
# cross_SE = empty_SE.copy()
# cross_SE[SE_rad-spoke_halfwid:SE_rad+1+spoke_halfwid, :] = True
# cross_SE[:, SE_rad-spoke_halfwid:SE_rad+1+spoke_halfwid] = True
# spokes_SE = cross_SE.copy()
# spokes_SE[SE_rad-4:SE_rad+5, SE_rad-4:SE_rad+5] = False
# center_SE = empty_SE.copy()
# center_SE[SE_rad-1:SE_rad+2, SE_rad-1:SE_rad+2] = True
# corners_SE = np.logical_not(cross_SE)
# corners_SE[SE_rad-4:SE_rad+5, SE_rad-4:SE_rad+5] = False

# xx=183
# yy = 173
# test = img[yy-SE_rad:yy+SE_rad+1, xx-SE_rad:xx+SE_rad+1]

# plt.close('all')

# # plt.figure()
# # plt.subplot(1,4,1)
# # plt.imshow(img)
# # plt.axis('off')
# # plt.subplot(1,4,2)
# # plt.imshow(test)
# # plt.axis('off')
# # plt.subplot(1,4,3)
# # # plt.imshow(cross_SE)
# # plt.imshow(spokes_SE)
# # plt.axis('off')
# # plt.subplot(1,4,4)
# # plt.imshow(corners_SE)
# # plt.axis('off')



# glenna = np.zeros(img.shape)
# for xx in range(SE_rad,width-SE_rad):
#     for yy in range(SE_rad,height-SE_rad):
#         pixOI = img[yy,xx]
#         test = img[yy-SE_rad:yy+SE_rad+1, xx-SE_rad:xx+SE_rad+1]
#         spokes_mean = np.mean(test[spokes_SE])
#         center_mean = np.mean(test[center_SE])
#         corners_mean = np.mean(test[corners_SE])
        
#         if (center_mean-spokes_mean>8) and (corners_mean > spokes_mean):
#             glenna[yy,xx]= 1 #center_mean-spokes_mean
# #             plt.plot(yy,xx,'.r')

# print('done!')


done!
